In [1]:
from __future__ import absolute_import, division, print_function

import os
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata, metadata_io,dataset_schema 
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
tf.enable_eager_execution()

In [20]:
os.getcwd()

'/Users/Niklas/Documents/tf_ner/models'

# HAVE TO RE RUN ALL! AND CLEAR FOLDERS

https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

In [2]:
transformed_metadata = metadata_io.read_metadata(
    os.path.join(
      "gs://named_entity_recognition/beam/", transform_fn_io.TRANSFORMED_METADATA_DIR
      )
    )
transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

In [3]:
transformed_feature_spec

{u'chars': FixedLenFeature(shape=[30, 10], dtype=tf.int64, default_value=None),
 u'chars_in_word': FixedLenFeature(shape=[30], dtype=tf.int64, default_value=None),
 u'id': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 u'label': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'label_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'labels': FixedLenFeature(shape=[30], dtype=tf.string, default_value=None),
 u'sentence_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'text': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'word_representation': VarLenFeature(dtype=tf.int64)}

In [4]:
def _read_from_tfrecord(example_proto):
    features = tf.parse_example([example_proto], features=transformed_feature_spec)

    return features

In [5]:
ds = tf.data.TFRecordDataset("gs://named_entity_recognition/beam/TEST-00000-of-00002").map(_read_from_tfrecord).batch(2)
for x in ds:
    break


In [6]:
from string import ascii_lowercase
MAPPING = {a:index for index,a in enumerate(ascii_lowercase + ascii_lowercase.upper())}

In [7]:
x["text"]

<tf.Tensor: id=84, shape=(2, 1), dtype=string, numpy=
array([['The London march came ahead of anti-war protests today in other cities , including Rome , Paris , and Madrid .'],
       ['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .']],
      dtype=object)>

In [8]:
SENTENCE_LEN=30
WORD_LEN=10
BATCH_SIZE=2
CHAR_SIZE=54
EMBED_DIM_CHAR=int(CHAR_SIZE**(1/4))*8
DEFAULT_WORD_VALUE =0
VOCAB_SIZE=1e4
EMBED_DIM_WORD=int(VOCAB_SIZE**(1/4))*8
input_char = tf.reshape(x["chars"],[BATCH_SIZE,SENTENCE_LEN,WORD_LEN])
char_embedding = tf.contrib.layers.embed_sequence(
    input_char, vocab_size=CHAR_SIZE, embed_dim=EMBED_DIM_CHAR)

In [9]:
# No padding needed now :) 
input_word = tf.sparse_to_dense(x["word_representation"].indices,[BATCH_SIZE,1,SENTENCE_LEN],
                           x["word_representation"].values,default_value=DEFAULT_WORD_VALUE
                          )
input_word = tf.reshape(input_word,[BATCH_SIZE,SENTENCE_LEN])
word_embedding = tf.contrib.layers.embed_sequence(
    input_word, vocab_size=VOCAB_SIZE, embed_dim=EMBED_DIM_WORD)

In [10]:
# Print the shapes 
print(tf.shape(input_word))
print(tf.shape(word_embedding))
print(tf.shape(char_embedding))

tf.Tensor([ 2 30], shape=(2,), dtype=int32)
tf.Tensor([ 2 30 80], shape=(3,), dtype=int32)
tf.Tensor([ 2 30 10 16], shape=(4,), dtype=int32)


In [11]:
s = tf.shape(char_embedding)
char_embeddings = tf.reshape(char_embedding, shape=[-1, s[-2], s[-1]])
word_lengths = tf.reshape(x['chars_in_word'],[-1])

In [12]:
CHAR_HIDDEN_SIZE=10
# 3. bi lstm on chars
cell_fw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)
cell_bw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)

_, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, char_embeddings, sequence_length=word_lengths,
    dtype=tf.float32)
#shape = (batch x sentence, 2 x char_hidden_size)
output = tf.concat([output_fw, output_bw], axis=-1)

# shape = (batch, sentence, 2 x char_hidden_size)
char_rep = tf.reshape(output, shape=[-1, s[1], 2*CHAR_HIDDEN_SIZE])

# shape = (batch, sentence, 2 x char_hidden_size + word_vector_size)
word_embeddings = tf.concat([word_embedding, char_rep], axis=-1)

In [13]:
# WORD LSTM:S

HIDDEN_SIZE=20
cell_fw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
cell_bw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
sequence_length= tf.reshape(x['sentence_length'],[BATCH_SIZE])
(output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, word_embeddings, sequence_length=sequence_length,
    dtype=tf.float32)
context_rep = tf.concat([output_fw, output_bw], axis=-1)

In [14]:
sequence_length

<tf.Tensor: id=1053, shape=(2,), dtype=int64, numpy=array([21, 24])>

In [15]:
tf.shape(context_rep)

<tf.Tensor: id=3463, shape=(3,), dtype=int32, numpy=array([ 2, 30, 40], dtype=int32)>

In [16]:
CLASSES_OUTPUT=3

ntime_steps = tf.shape(context_rep)[1]
context_rep_flat = tf.reshape(context_rep, [-1, 2*HIDDEN_SIZE])
preds = tf.layers.dense(inputs = context_rep_flat,units=CLASSES_OUTPUT)
logits = tf.reshape(preds, [-1, ntime_steps, CLASSES_OUTPUT])


In [17]:
crf_params = tf.get_variable("crf", [CLASSES_OUTPUT, CLASSES_OUTPUT], dtype=tf.float32)
pred_ids, _ = tf.contrib.crf.crf_decode(logits, crf_params, sequence_length)

In [18]:
LOOK_UP_TABLE="FILE_NAME_TO_LOOP_UP_VOCAB"

if mode == tf.estimator.ModeKeys.PREDICT:
    # Predictions
    # GET THE WORD FORM THE ID, LINK THIS WITH THE BEAM TRANSFORM VOCAB
    reverse_vocab_tags = tf.contrib.lookup.index_to_string_table_from_file(
        LOOK_UP_TABLE)
    pred_strings = reverse_vocab_tags.lookup(tf.to_int64(pred_ids))
    predictions = {
        'pred_ids': pred_ids,
        'tags': pred_strings
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
else:
    # Loss
    tags = labes
    log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, tags, nwords, crf_params)
    loss = tf.reduce_mean(-log_likelihood)

    # Metrics
    # AWESOME THIS IS GREAT TO USE THIS MASK HERE
    # WILL SOLVE A LOT OF PROBLEMS, OTHERWISE
    # GOOD LEARNING AS WELL
    weights = tf.sequence_mask(nwords)
    metrics = {
        'acc': tf.metrics.accuracy(tags, pred_ids, weights),
        'precision': precision(tags, pred_ids, num_tags, indices, weights),
        'recall': recall(tags, pred_ids, num_tags, indices, weights),
        'f1': f1(tags, pred_ids, num_tags, indices, weights),
    }
    # ADD THEM ALL TO THE GRAPH
    for metric_name, op in metrics.items():
        tf.summary.scalar(metric_name, op[1])
    
    # WHAT WILL HAPPEN IF WE HAVE EVAL MODE
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # WHAT WILL HAPPEN IF WE HAVE TRAIN? 
    elif mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer().minimize(
            loss, global_step=tf.train.get_or_create_global_step())
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, train_op=train_op)

SyntaxError: 'return' outside function (<ipython-input-18-825a1a96e791>, line 13)